### Tipos de Testes de Hipóteses

Testes de hipótese servem para verificar, se uma afirmação sobre uma população é plausível ou deve ser rejeitada. Eles auxiliam a tomada de decisões com base em dados.

- Testes de Normalidade
- Testes Paramétricos
    - Usados quando os dados seguem a **Distribuição Normal**.

- Testes Não Paramétricos
    - Usados quando os dados são **Ordinais** ou não seguem a **Distribuição Normal**
- Teste Bayesianos
- Testes de Regressão

In [2]:
""" Imports """

import pandas as pd
import numpy as np
import math

import plotly.express as px
import plotly.graph_objects as go

from scipy import stats
from scipy.stats import norm, shapiro, kstest, pearsonr, linregress, mannwhitneyu

### Testes de Normalidade

In [2]:
""" Datasets randomico para ser usado nos testes de normalidade para Distribuição Normal. """

dados_distr_normal = np.random.normal(loc=0, scale=1, size=100)
dados_distr_normal

array([-1.07825809, -1.16370496,  0.13232924, -0.85216077,  0.2608951 ,
       -0.39468379,  0.70920589, -0.74956437, -0.36417351,  0.37268835,
       -0.38573863, -0.37214809, -0.7657644 ,  1.83647905, -0.93537661,
        0.4102178 ,  1.24415943, -1.91086398, -1.27350585,  0.71845578,
        0.16678922, -0.97620174,  0.34213856, -1.64512429, -0.76708032,
        0.1223888 ,  0.76599637,  1.05205127,  0.80248843, -0.14907527,
       -0.07710566,  0.07483199, -0.86475147,  0.0727895 ,  0.10783397,
        1.62299658,  1.11607952, -0.48368251, -0.23891463,  0.14672219,
       -0.21238485,  0.20441925,  0.35312045,  0.59837192,  0.55353746,
       -1.02971909,  0.37534581, -0.40835697,  1.25719482,  1.29557315,
        0.16854979,  1.3946269 ,  0.98621252, -0.19097569, -0.77698288,
        1.92850504,  0.01985481,  0.41622566,  0.22695478,  1.91684769,
       -1.14843595, -1.42871268,  1.45167744,  0.51285411,  0.06822467,
       -0.36634177, -0.19849779, -2.05406021, -2.339248  , -2.06

In [3]:
""" Gráfico da curva normal com base nos dados obtidos. """

def grafico_distribuicao(data):
    # Criar curva normal
    x = np.linspace(min(data), max(data), 200)
    y = norm.pdf(x, np.mean(data), np.std(data))
    media = np.mean(data)

    df = pd.DataFrame({"x": x, "densidade": y})

    # Criar gráfico
    fig = px.line(df, x="x", y="densidade", title="Curva de Distribuição")
    # Adicionar a linha da média
    fig.add_vline(x=media, line_dash="dash", line_color="red", annotation_text=f"Média = {media:.2f}")

    fig.show()

In [4]:
grafico_distribuicao(dados_distr_normal)

In [5]:
""" Datasets randomico para ser usado nos testes de normalidade para Distribuição Não Normal. """

dados_distr_nao_normal = np.random.exponential(scale=1, size=100)
dados_distr_nao_normal

array([0.37327606, 0.26546045, 2.04471402, 0.41420633, 0.56592218,
       0.89172848, 1.2712314 , 0.75597237, 0.11847101, 0.86106984,
       1.8364916 , 3.29012412, 1.97822121, 0.11452164, 0.64205698,
       0.07703847, 0.96768498, 0.22012445, 0.25065835, 1.28980041,
       2.004714  , 0.80953915, 1.85144332, 0.88577513, 0.35978207,
       1.72964688, 0.99097684, 3.31888318, 0.29690178, 0.52462724,
       1.99537477, 0.25019893, 2.38041507, 0.04617642, 0.2436035 ,
       0.04722772, 0.29650267, 0.67822398, 0.44692751, 1.92527218,
       0.15410038, 0.23375301, 2.20319336, 1.3106156 , 1.72047617,
       1.25138763, 0.06580673, 0.06071135, 0.20370964, 0.09536913,
       2.08453164, 0.26573101, 2.07871593, 3.50403999, 2.14391277,
       0.16836924, 2.40400454, 0.14930702, 2.48758738, 3.22126726,
       0.50582753, 1.82047433, 0.305781  , 2.27615361, 0.17920223,
       0.48911582, 0.55703111, 1.43281949, 0.51994556, 0.78932014,
       0.37498884, 2.11343358, 0.27490098, 0.43332905, 2.84765

In [6]:
grafico_distribuicao(dados_distr_nao_normal)

Os **Testes de Normalidade** servem para verificar se um conjunto de dados seguem uma **Distribuição Normal** (se os dados se concentram em torno da média)

Sendo os tipos de testes de normalidade:

- **Shapiro-Wilk**
- **Kolmogorov-Smirnov**
- Anderson-Darling
- Jaque-Bera

#### Shapiro Wilk

Serve para verificar se um conjunto de dados **segue uma distribuição normal**.  
É especialmente indicado para **amostras pequenas ou médias**.

**Fórmula:**
$$
W = \frac{\left(\sum_{i=1}^{n} a_i x_{(i)}\right)^2}
{\sum_{i=1}^{n} (x_i - \bar{x})^2}
$$

- x_(i): valores da amostra ordenados em ordem crescente  
- xᵢ: valores observados  
- x̄: média da amostra  
- aᵢ: constantes calculadas a partir da distribuição normal teórica  
- n: tamanho da amostra  

**Resultado:**
- **p ≥ 0,05** → os dados **podem seguir** uma distribuição normal  
- **p < 0,05** → os dados **provavelmente não seguem** uma distribuição normal

In [7]:
""" Função para teste de normalidade usando Shapiro Wilk """

def testar_normalidade_shapiro(data):
    stat, p = shapiro(data)

    print(f"Estatística W = {stat:.4f}") # Similaridade dos dados com uma normal, sendo 1 = perfeitamente normal
    print(f"p-valor = {p:.4f}")

    if p > 0.05:
        print("✔ A distribuição pode ser considerada normal (falha em rejeitar H0).")
    else:
        print("✘ A distribuição NÃO é normal (rejeita H0).")

In [8]:
testar_normalidade_shapiro(dados_distr_normal)

Estatística W = 0.9889
p-valor = 0.5765
✔ A distribuição pode ser considerada normal (falha em rejeitar H0).


In [9]:
testar_normalidade_shapiro(dados_distr_nao_normal)

Estatística W = 0.8897
p-valor = 0.0000
✘ A distribuição NÃO é normal (rejeita H0).


#### Kolmogorov-Smirnov

Compara a distribuição dos seus dados com **qualquer distribuição teórica**
(normal, exponencial, uniforme, etc.).

É **menos sensível ao tamanho da amostra** do que o teste de Shapiro–Wilk
quando usado para normalidade.

**Fórmula:**
$$
D = \sup_x \left| F_n(x) - F(x) \right|
$$

- $F_n(x)$: função de distribuição empírica da amostra  
- $F(x)$: função de distribuição acumulada da distribuição teórica escolhida  
- $\sup$: maior diferença absoluta observada  
- $D$: estatística do teste Kolmogorov–Smirnov  

**Interpretação do resultado**
- **p ≥ 0,05** → os dados **combinam** com a distribuição escolhida  
- **p < 0,05** → os dados **não seguem** a distribuição escolhida


In [10]:
""" Função para teste de normalidade usando Kolmogorov-Smirnov """

def testar_normalidade_ks(data):
    media = np.mean(data)
    desvio = np.std(data, ddof=1)

    # Teste KS comparando com uma normal teórica
    stat, p = kstest(data, 'norm', args=(media, desvio))

    print(f"Estatística KS = {stat:.4f}") # Maior diferença entre distribuições acumuladas, sendo 0 = distribuições identicas
    print(f"p-valor = {p:.4f}")

    if p > 0.05:
        print("✔ A distribuição pode ser considerada normal (falha em rejeitar H0).")
    else:
        print("✘ A distribuição NÃO é normal (rejeita H0).")

In [11]:
testar_normalidade_ks(dados_distr_normal)

Estatística KS = 0.0717
p-valor = 0.6567
✔ A distribuição pode ser considerada normal (falha em rejeitar H0).


In [12]:
testar_normalidade_ks(dados_distr_nao_normal)

Estatística KS = 0.1533
p-valor = 0.0162
✘ A distribuição NÃO é normal (rejeita H0).


### Testes Paramétricos

Para realizar um teste paramétrico, é necessário:
- Que os valores sigam uma distribuição normal.
- Sejam valores quantitativos (intervalar ou razão).

Sendo os tipos de testes paramétricos:
- Teste Z
- Teste t de Student
- Anova
- Correlação de Person
- Regressão Linear

---

#### Teste Z

O Teste Z é um teste estatístico paramétrico utilizado para avaliar se a média de uma amostra — ou uma proporção amostral — é estatisticamente diferente de um valor previamente conhecido da população.

Esse teste é indicado quando o desvio padrão populacional é conhecido e o tamanho da amostra é grande (geralmente n ≥ 30).

**Teste para as Médias**  
Utilizado quando se analisa o valor médio de uma variável numérica:
$$
Z = \frac{\bar{x} - \mu}{\sigma / \sqrt{n}}
$$

**Teste Z para as Proporções**  
Utilizado quando se analisa a fração de elementos que possuem uma determinada característica em relação ao total da amostra:
$$
Z = \frac{\hat{p} - p}{\sqrt{\frac{p(1 - p)}{n}}}
$$

- p < 0.05 → Rejeita-se a hipótese nula (H₀)
- p ≥ 0.05 → Não se rejeita a hipótese nula (H₀)

In [13]:
def teste_z_media_plot(amostra, mu, sigma, alpha=0.05):

    amostra = np.array(amostra)
    n = len(amostra)
    media = amostra.mean()

    z = (media - mu) / (sigma / np.sqrt(n))
    p_valor = 2 * (1 - norm.cdf(abs(z)))

    # Gráfico
    x = np.linspace(-4, 4, 400)
    y = norm.pdf(x)
    z_critico = norm.ppf(1 - alpha / 2)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Normal'))

    fig.add_trace(go.Scatter(
        x=[z, z], y=[0, norm.pdf(z)],
        mode='lines', name=f'Z observado = {z:.2f}'
    ))

    fig.add_trace(go.Scatter(
        x=x[x >= z_critico], y=y[x >= z_critico],
        fill='tozeroy', opacity=0.3, name='Região crítica'
    ))

    fig.add_trace(go.Scatter(
        x=x[x <= -z_critico], y=y[x <= -z_critico],
        fill='tozeroy', opacity=0.3, showlegend=False
    ))

    fig.update_layout(
        title=f"Teste Z clássico (z={z:.2f}, p={p_valor:.4f})",
        xaxis_title="Z",
        yaxis_title="Densidade"
    )

    fig.show()
    decisao = "Rejeita H₀" if p_valor < alpha else "Não rejeita H₀"
    return z, p_valor, decisao

In [14]:
dados = [52, 50, 51, 53, 54, 49, 50, 52, 51, 53]

z, p, decisao = teste_z_media_plot(dados, mu=50, sigma=2)

print("Z:", z)
print("p-valor:", p)
print("Decisão:", decisao)

Z: 2.3717082451262845
p-valor: 0.017706065807366667
Decisão: Rejeita H₀


In [15]:
def teste_z_proporcao_plot(sucessos, n, p0, alpha=0.05):
    """
    sucessos : número de sucessos observados
    n        : tamanho da amostra
    p0       : proporção populacional (H0)
    alpha    : nível de significância
    """

    # Proporção amostral
    p_hat = sucessos / n

    # Estatística Z
    z = (p_hat - p0) / np.sqrt(p0 * (1 - p0) / n)

    # p-valor (bicaudal)
    p_valor = 2 * (1 - norm.cdf(abs(z)))

    # Curva normal
    x = np.linspace(-4, 4, 400)
    y = norm.pdf(x)

    # Valor crítico
    z_critico = norm.ppf(1 - alpha / 2)

    fig = go.Figure()

    # Distribuição normal
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        name='Distribuição Normal'
    ))

    # Linha do Z observado
    fig.add_trace(go.Scatter(
        x=[z, z],
        y=[0, norm.pdf(z)],
        mode='lines',
        name=f'Z observado = {z:.2f}'
    ))

    # Regiões críticas
    fig.add_trace(go.Scatter(
        x=x[x >= z_critico],
        y=y[x >= z_critico],
        fill='tozeroy',
        name='Região crítica',
        opacity=0.3
    ))

    fig.add_trace(go.Scatter(
        x=x[x <= -z_critico],
        y=y[x <= -z_critico],
        fill='tozeroy',
        showlegend=False,
        opacity=0.3
    ))

    fig.update_layout(
        title=f"Teste Z para Proporções (z={z:.2f}, p={p_valor:.4f})",
        xaxis_title="Z",
        yaxis_title="Densidade"
    )
    fig.show()
    decisao = "Rejeita H₀" if p_valor < alpha else "Não rejeita H₀"
    return z, p_valor, decisao

In [16]:
z, p, decisao = teste_z_proporcao_plot(sucessos=60, n=100, p0=0.5)

print("Z:", z)
print("p-valor:", p)
print("Decisão:", decisao)

Z: 1.9999999999999996
p-valor: 0.04550026389635842
Decisão: Rejeita H₀


#### Teste t de Student

O **Teste t de Student** é um teste estatístico utilizado para comparar **médias**, especialmente quando o tamanho da amostra é pequeno (geralmente **𝑛 < 30**) e o **desvio padrão** populacional é **desconhecido**.

Ele avalia se a diferença entre a média observada e a média hipotética é estatisticamente significativa ou se pode ser atribuída ao acaso.

Fórmula:
$$
t = \frac{\bar{x} - \mu}{s / \sqrt{n}}
$$

- p < 0.05 → Rejeita-se a hipótese nula (H₀)
- p ≥ 0.05 → Não se rejeita a hipótese nula (H₀)

In [17]:
def teste_t_uma_amostra(amostra, mu=0, alpha=0.05):
    n = len(amostra)
    media = sum(amostra) / n
    variancia = sum((x - media) ** 2 for x in amostra) / (n - 1)
    s = math.sqrt(variancia)

    t = (media - mu) / (s / math.sqrt(n))
    p = 2 * (1 - stats.t.cdf(abs(t), df=n-1))

    decisao = "Rejeita H0" if p < alpha else "Não rejeita H0"

    return t, p, decisao

In [18]:
dados = [10, 12, 9, 11, 13, 10, 12]

t, p, decisao = teste_t_uma_amostra(dados, mu=11)

print(f"t = {t:.3f}")
print(f"p-valor = {p:.4f}")
print(decisao)

t = 0.000
p-valor = 1.0000
Não rejeita H0


#### Anova

O teste ANOVA (Analysis of Variance — Análise de Variância) é um teste estatístico utilizado para comparar as médias de três ou mais grupos simultaneamente.

Ele avalia se pelo menos uma das médias difere estatisticamente das demais, analisando a relação entre a variabilidade entre os grupos e a variabilidade dentro dos grupos.
Quando a variabilidade entre grupos é significativamente maior do que a variabilidade interna, conclui-se que nem todas as médias são iguais.

A estatística do teste é dada por:

$$
F = \frac{SS_{\text{entre}} / (k - 1)}{SS_{\text{dentro}} / (n - k)}
$$

Resultado:
- p < 0.05 → rejeita-se H₀  
Existe diferença estatisticamente significativa entre as médias

- p ≥ 0.05 → não se rejeita H₀  
Não há evidência suficiente de diferença entre as médias

In [19]:
def anova_um_fator(grupos, alpha=0.05):
    """
    Realiza o teste ANOVA de um fator.

    Parâmetros:
    grupos : lista de listas
             Cada lista interna representa um grupo
    alpha  : nível de significância

    Retorna:
    F       : estatística F
    p       : p-valor
    decisao : decisão do teste
    """

    F, p = stats.f_oneway(*grupos)
    decisao = "Rejeita H0" if p < alpha else "Não rejeita H0"
    return F, p, decisao

In [20]:
grupo1 = [10, 12, 9, 11]
grupo2 = [14, 15, 13, 16]
grupo3 = [9, 8, 10, 9]

F, p, decisao = anova_um_fator([grupo1, grupo2, grupo3])

print(f"F = {F:.3f}")
print(f"p-valor = {p:.4f}")
print(decisao)

F = 24.250
p-valor = 0.0002
Rejeita H0


#### Correlação de Person

A **correlação de Pearson** é uma medida estatística que avalia o **grau (força)** e a **direção** da **relação linear** entre duas variáveis **quantitativas**.

O valor de **r** indica a **força** e a **direção** da relação
- **r > 0**: relação linear positiva
- **r < 0**: relação linear negativa
- **r = 0**: ausência de relação linear

O **p-value** é usado para verificar se a correlação observada é estatisticamente **significativa**:
- **p < 0,05** → correlação estatisticamente significativa
- **p ≥ 0,05** → correlação não significativa

Fórmula da correlação de Pearson:
$$
r = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}
{\sqrt{\sum (x_i - \bar{x})^2 \sum (y_i - \bar{y})^2}}
$$

Interpretação do resultado:
- $r > 0$ → relação linear positiva
- $r < 0$ → relação linear negativa
- $r \approx 0$ → ausência de relação linear

Teste de significância:
- p ≥ 0,05 → correlação não estatisticamente significativa
- p < 0,05 → correlação estatisticamente significativa

In [21]:
def correlacao_pearson_plot(x, y):
    x = np.array(x)
    y = np.array(y)

    # Correlação de Pearson
    r, p_valor = pearsonr(x, y)

    # Regressão linear (y = ax + b)
    a, b = np.polyfit(x, y, 1)
    y_pred = a * x + b

    # Gráfico
    fig = go.Figure()

    # Pontos
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name='Dados'
    ))

    # Reta de regressão
    fig.add_trace(go.Scatter(
        x=x,
        y=y_pred,
        mode='lines',
        name=f'Reta (r={r:.2f})'
    ))

    fig.update_layout(
        title=f'Correlação de Pearson (r={r:.2f}, p={p_valor:.4f})',
        xaxis_title='X',
        yaxis_title='Y'
    )
    fig.show()

    if r > 0:
        print("Relação linear positiva")
    elif r < 0:
        print("Relação linear negativa")
    elif r == 0:
        print("Ausencia de relação linear")

    if p_valor >= 0.05:
        print("Coorrelação não significativa")
    else:
        print("Coorrelação significativa")

    return r, p_valor, a, b

In [22]:
x = [1, 2, 3, 4, 5]
y = [2, 4, 5, 8, 10]

r, p, a, b = correlacao_pearson_plot(x, y)

print("r:", r)
print("p-valor:", p)
print("reta: y =", a, "* x +", b)

Relação linear positiva
Coorrelação significativa
r: 0.9901475429766743
p-valor: 0.00117221644471691
reta: y = 2.0 * x + -0.19999999999999896


#### Regressão Linear

Regressão linear é um método estatístico usado para modelar e testar a relação entre uma variável dependente (Y) e uma ou mais variáveis independentes (X), assumindo que essa relação pode ser aproximada por uma linha reta.

Fórmula:
$$
Y = \beta_0 + \beta_1 X + \varepsilon
$$

- Y = variável dependente  
- X = variável independente  
- β₀ = intercepto  
- β₁ = coeficiente angular  
- ε = erro aleatório  

Forma estimada do modelo:
$$
\hat{Y} = b_0 + b_1 X
$$

Resultado:  
- H₀ (hipótese nula): o coeficiente da regressão = 0 (X não tem efeito sobre Y)  
- H₁ (hipótese alternativa): o coeficiente ≠ 0 (X tem efeito sobre Y)  

In [23]:
def teste_hipotese_regressao_scipy(x, y, alpha=0.05):
    resultado = linregress(x, y)

    coef = resultado.slope
    intercepto = resultado.intercept
    p_valor = resultado.pvalue
    decisao = "Rejeitar H0" if p_valor < alpha else "Falha em rejeitar H0"

    # Criando gráfico Plotly
    y_pred = [intercepto + coef * xi for xi in x]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Dados'))
    fig.add_trace(go.Scatter(x=x, y=y_pred, mode='lines', name='Regressão', line=dict(color='red')))
    fig.update_layout(
        title=f"Regressão Linear: coef={coef:.3f}, p-valor={p_valor:.3f}",
        xaxis_title="X",
        yaxis_title="Y",
        template="plotly_white"
    )

    return {
        "coeficiente": coef,
        "intercepto": intercepto,
        "p_valor": p_valor,
        "decisao": decisao,
        "figura": fig
    }

In [24]:
# Exemplo de uso
x = [1, 2, 3, 4, 5]
y = [2, 4, 5, 4, 5]

resultado = teste_hipotese_regressao_scipy(x, y)
print(resultado["coeficiente"], resultado["p_valor"], resultado["decisao"])
resultado["figura"].show()

0.6000000000000001 0.12402706265755442 Falha em rejeitar H0


### Testes Não Paramétricos

Testes não paramétricos são testes estatísticos que **não exigem pressupostos rigorosos sobre a distribuição dos dados**, como normalidade ou aderência a uma distribuição conhecida.

Eles são geralmente utilizados quando os dados apresentam as seguintes características:

- Não seguem uma distribuição normal
- Contêm outliers
- Possuem amostras pequenas
- São dados ordinais (por exemplo, escalas de satisfação)

Alguns exemplos de testes não paramétricos são:

- Mann–Whitney U
- Wilcoxon pareado
- Kruskal–Wallis
- Friedman
- Qui-quadrado
- Correlação de Spearman


#### Mann-Whitney U

O **teste Mann–Whitney U** é utilizado para comparar **dois grupos independentes**, com o objetivo de verificar se eles pertencem à **mesma distribuição** ou se existe **diferença estatisticamente significativa** entre eles.  

Ele é considerado a alternativa não paramétrica ao **teste t para amostras independentes**.

---

**Estatístico do teste**

Sejam:

- `n1` = tamanho da amostra do grupo 1  
- `n2` = tamanho da amostra do grupo 2  
- `R1` = soma dos postos (ranks) do grupo 1  
- `R2` = soma dos postos (ranks) do grupo 2  

Os estatísticos são definidos por:

$$
U_1 = n_1 n_2 + \frac{n_1 (n_1 + 1)}{2} - R_1
$$

$$
U_2 = n_1 n_2 + \frac{n_2 (n_2 + 1)}{2} - R_2
$$

O valor utilizado no teste é:

$$
U = \min(U_1, U_2)
$$

---

**Aproximação normal (amostras grandes)**

Quando `n1 ≥ 20` e `n2 ≥ 20`, o estatístico \( U \) pode ser aproximado por uma **distribuição normal**.

**Média e variância de \( U \)**

$$
\mu_U = \frac{n_1 n_2}{2}
$$

$$
\sigma_U = \sqrt{\frac{n_1 n_2 (n_1 + n_2 + 1)}{12}}
$$

**Estatístico Z**

$$
Z = \frac{U - \mu_U}{\sigma_U}
$$

---

**Hipóteses do teste**

- **Hipótese nula (H₀):** as duas amostras provêm da mesma distribuição  
- **Hipótese alternativa (H₁):** as distribuições são diferentes  

---

**Observação:**  
Em casos de empates nos dados, é necessária uma correção na variância do estatístico. Essa correção é aplicada automaticamente por softwares estatísticos.

In [ ]:
def teste_mann_whitney(grupo1, grupo2, alpha=0.05):
    """
    Executa o teste Mann–Whitney U usando SciPy.

    Parâmetros:
    - grupo1, grupo2: listas ou arrays com os dados
    - alpha: nível de significância (padrão = 0.05)

    Retorna:
    - dicionário com U, p-valor e decisão do teste
    """

    u_stat, p_value = mannwhitneyu(
        grupo1,
        grupo2,
        alternative="two-sided"  # teste bilateral
    )

    decisao = (
        "Rejeita H₀: há diferença significativa entre os grupos"
        if p_value < alpha
        else "Não rejeita H₀: não há diferença significativa entre os grupos"
    )

    return {
        "U": u_stat,
        "p_value": p_value,
        "alpha": alpha,
        "decisao": decisao
    }

In [4]:
grupo_a = [5, 7, 8, 6, 9]
grupo_b = [3, 4, 6, 2, 5]

resultado = teste_mann_whitney(grupo_a, grupo_b)

print("Resultado do teste Mann–Whitney U")
print(f"U = {resultado['U']}")
print(f"p-valor = {resultado['p_value']:.4f}")
print(resultado["decisao"])

Resultado do teste Mann–Whitney U
U = 23.0
p-valor = 0.0356
Rejeita H₀: há diferença significativa entre os grupos


#### Wilcoxon pareado

In [ ]:
pass

#### Kruskal-Wallis

In [ ]:
pass

#### Friedman

In [ ]:
pass

#### Qui-quadrado

In [ ]:
pass

#### Correlação de Spearman

In [ ]:
pass